In [45]:
from typing import Literal

import numpy as np
import torch
from torch_geometric.loader import DataLoader
from torch_scatter import scatter_max
from tqdm import tqdm


from geometric_governance.util import Logger, RangeOrValue, get_value
from geometric_governance.data import (
    generate_synthetic_election,
    get_scoring_function_winners,
)
from geometric_governance.model import MessagePassingElectionModel, DeepSetStrategyModel

device = torch.device(0) if torch.cuda.is_available() else torch.device("cpu")

config = {
    "num_voters_range": (3, 50),
    "num_candidates_range": (2, 10),
    "train_dataset_size": 1000,
    "train_batch_size": 128,
    "train_num_epochs": 20,
    "eval_num_voters": 75,
    "eval_num_candidates": 15,
    "eval_dataset_size": 1_000,
    "welfare_fn": "nash",
    "top_k_candidates": None,
    "learning_rate": 3e-4,
    "use_welfare_loss": True,
    "use_monotonicity_loss": True,
    "monotonicity_loss_batch_size": 32,
    "strategy_loss_batch_size": 64,
}


def generate_welfare_bipartite_dataset(
    dataset_size: int,
    num_voters_range: RangeOrValue,
    num_candidates_range: RangeOrValue,
    dataloader_batch_size: int,
    top_k_candidates: int,
    welfare_fn: Literal["utilitarian", "nash", "rawlsian"],
    rng: np.random.Generator,
):
    graphs = []
    for _ in tqdm(range(dataset_size)):
        num_voters = get_value(num_voters_range, rng)
        num_candidates = get_value(num_candidates_range, rng)

        election_data = generate_synthetic_election(
            num_voters=num_voters, num_candidates=num_candidates, rng=rng
        )

        graph = election_data.to_bipartite_graph(top_k_candidates, vote_data="ranking")

        candidate_welfare = election_data.voter_utilities
        match welfare_fn:
            case "utilitarian":
                candidate_welfare = election_data.voter_utilities.sum(dim=0)
            case "nash":
                #print(election_data.voter_utilities[:, 0])
                log_voter_utilities = torch.log(election_data.voter_utilities)
                #print(log_voter_utilities[:, 0])
                candidate_welfare = log_voter_utilities.sum(dim=0)
                #print(candidate_welfare[0])
            case "rawlsian":
                candidate_welfare = election_data.voter_utilities.min(dim=0)[0]
            case _:
                raise ValueError("Unknown welfare function.")

        winners = get_scoring_function_winners(candidate_welfare)

        graph.y = candidate_welfare
        graph.winners = winners
        graphs.append(graph)
    dataloader = DataLoader(graphs, batch_size=dataloader_batch_size, shuffle=True)
    return dataloader


train_dataloader = generate_welfare_bipartite_dataset(
    dataset_size=config["train_dataset_size"],
    num_voters_range=config["num_voters_range"],
    num_candidates_range=config["num_candidates_range"],
    dataloader_batch_size=config["train_batch_size"],
    top_k_candidates=config["top_k_candidates"],
    welfare_fn=config["welfare_fn"],
    rng=np.random.default_rng(seed=42),
)

eval_dataloader = generate_welfare_bipartite_dataset(
    dataset_size=config["eval_dataset_size"],
    num_voters_range=config["eval_num_voters"],
    num_candidates_range=config["eval_num_candidates"],
    dataloader_batch_size=config["train_batch_size"],
    top_k_candidates=config["top_k_candidates"],
    welfare_fn=config["welfare_fn"],
    rng=np.random.default_rng(seed=16180),
)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 299.50it/s]


In [52]:
edge_dim = 1

strategy_model = DeepSetStrategyModel(edge_dim=edge_dim, emb_dim=32)
election_model = MessagePassingElectionModel(edge_dim=edge_dim)
election_model.load_state_dict(torch.load("election_model", weights_only=True))

strategy_model.to(device=device)
election_model.to(device=device)

s_optim = torch.optim.Adam(strategy_model.parameters(), lr=config["learning_rate"])
e_optim = torch.optim.Adam(election_model.parameters(), lr=config["learning_rate"])

experiment_name = "robust-voting"
logger = Logger(
    experiment_name=experiment_name,
    config=config,
    mode="online",
)

with tqdm(range(config["train_num_epochs"])) as pbar:
    for epoch in range(config["train_num_epochs"]):
        # Train
        train_loss = 0
        train_welfare_loss = 0
        train_monotonicity_loss = 0
        train_welfare = 0

        election_model.train()
        for data_ in train_dataloader:
            data = data_.to(device=device)

            # Train strategy model
            strategic_votes = strategy_model(
                data.edge_attr, data.edge_index, data.candidate_idxs
            )

            strategy_loss = 0

            voters = (~data.candidate_idxs).nonzero()
            candidates_to_batch = data.batch[data.candidate_idxs.nonzero()]

            perm = torch.randperm(voters.size(0))[: config["strategy_loss_batch_size"]]
            for i in perm:
                # TODO: We can vectorise this by picking one random vote from each batch
                voter_idx = voters[i].item()
                batch = data.batch[voter_idx]

                voter_edge_attr_index = data.edge_index[0] == voter_idx

                gradient_mask = torch.ones(data.edge_attr.shape[0], device=device)
                gradient_mask[voter_edge_attr_index] = 0
                gradient_mask = gradient_mask.bool()
                gradient_mask = gradient_mask.unsqueeze(-1)

                # Pass through frozen election model
                gradient_cut_strategic_votes = torch.where(
                    gradient_mask, strategic_votes.detach(), strategic_votes
                )
                data_strategy = data.clone()
                data_strategy.edge_attr = strategic_votes
                out = election_model(data_strategy)[
                    (candidates_to_batch == batch).squeeze()
                ]

                # Calculate loss
                vote_probabilities = torch.exp(out)
                voter_welfare = data.edge_attr[voter_edge_attr_index].squeeze()

                loss = -(vote_probabilities * voter_welfare).sum()
                strategy_loss += loss

            strategy_loss.backward()
            s_optim.step()

            logger.log({
                "train/strategy_loss": strategy_loss,
            })

            ##########################

        #     # Train election model
        #     e_optim.zero_grad()

        #     with torch.no_grad():
        #         # strategic_votes = strategy_model(
        #         #     data.edge_attr, data.edge_index, data.candidate_idxs
        #         # )
        #         strategic_votes = data.edge_attr
        #     data.edge_attr = strategic_votes
        #     data.edge_attr.requires_grad = True
        #     out = election_model(data)

        #     if config["use_welfare_loss"]:
        #         welfare_loss = (
        #             -(torch.exp(out) * data.y).sum() / config["train_batch_size"]
        #         )
        #     else:
        #         welfare_loss = -(out * data.winners).sum() / config["train_batch_size"]

        #     # Monotonicity loss
        #     monotonicity_loss = 0

        #     if config["use_monotonicity_loss"]:
        #         candidates = data.candidate_idxs.nonzero()
        #         perm = torch.randperm(candidates.size(0))[
        #             : config["monotonicity_loss_batch_size"]
        #         ]
        #         for i in perm:
        #             candidate_idx = candidates[i]
        #             edge_idxs = data.edge_index[1] == candidate_idx
        #             grad = torch.autograd.grad(
        #                 outputs=out[i], inputs=data.edge_attr, create_graph=True
        #             )[0]
        #             monotonicity_loss += torch.where(
        #                 grad[edge_idxs] < 0,
        #                 -grad[edge_idxs],
        #                 torch.zeros_like(grad[edge_idxs]),
        #             ).mean()
        #     monotonicity_loss /= config["monotonicity_loss_batch_size"]

        #     loss = welfare_loss + monotonicity_loss
        #     loss.backward()
        #     e_optim.step()

        #     batch_idxs = data.batch[data.candidate_idxs]
        #     _, predicted = scatter_max(out, batch_idxs)
        #     welfare = data.y[predicted].mean()
        #     train_welfare += welfare.item()

        #     train_loss += loss.item()
        #     train_welfare_loss += welfare_loss.item()
        #     if config["use_monotonicity_loss"]:
        #         train_monotonicity_loss += monotonicity_loss.item()

        # train_loss /= len(train_dataloader)
        # train_welfare_loss /= len(train_dataloader)
        # train_monotonicity_loss /= len(train_dataloader)
        # train_welfare /= len(train_dataloader)

        # logger.log(
        #     {
        #         "train/total_loss": train_loss,
        #         "train/welfare_loss": train_welfare_loss,
        #         "train/monotonicity_loss": train_monotonicity_loss,
        #         "train/welfare": train_welfare,
        #     }
        # )

        # Eval
        election_model.eval()
        eval_loss = 0
        eval_welfare = 0
        total, correct = 0, 0
        with torch.no_grad():
            for data_ in eval_dataloader:
                data = data_.to(device=device)
                out = election_model(data)
                if config["use_welfare_loss"]:
                    welfare_loss = (
                        -(torch.exp(out) * data.y).sum() / config["train_batch_size"]
                    )
                else:
                    welfare_loss = (
                        -(out * data.winners).sum() / config["train_batch_size"]
                    )
                batch_idxs = data.batch[data.candidate_idxs]
                _, predicted = scatter_max(out, batch_idxs)
                _, predicted_ground = scatter_max(data.y, batch_idxs)
                total += predicted_ground.shape[0]
                correct += (predicted == predicted_ground).sum().item()

                welfare = data.y[predicted].mean()
                eval_loss += loss.item()
                eval_welfare += welfare.item()

        eval_loss /= len(eval_dataloader)
        eval_welfare /= len(eval_dataloader)
        eval_accuracy = correct / total
        logger.log(
            {
                "eval/loss": eval_loss,
                "eval/accuracy": eval_accuracy,
                "eval/welfare": eval_welfare,
            }
        )
        logger.commit()

        pbar.set_postfix(
            {
                "train_strategy_loss": strategy_loss,
                "train_welfare_loss": train_welfare_loss,
                "train_monotonicity_loss": train_monotonicity_loss,
                "train_welfare": train_welfare,
                "eval_loss": eval_loss,
                "eval_accuracy": eval_accuracy,
                "eval_welfare": eval_welfare,
            }
        )
        pbar.update(1)

100%|█| 20/20 [16:43<00:00, 50.19s/it, train_strategy_loss=tensor(-37.8427, grad_fn=<AddBackward0>), train_welfare_loss=0, train_monotonicity_loss=0, train_welfare=0, eval_loss=-0.605, ev


In [53]:
from itertools import islice

In [54]:
strategy_model.eval()
election_model.eval()

MessagePassingElectionModel(
  (lin_in_node): Linear(in_features=2, out_features=32, bias=True)
  (lin_out_node): Linear(in_features=32, out_features=1, bias=True)
  (lin_in_edge): Linear(in_features=1, out_features=8, bias=True)
  (convs): ModuleList(
    (0-3): 4 x MessagePassingElectionLayer()
  )
)

In [49]:
for data in islice(eval_dataloader, 1):
    print(data)
    print(data.edge_attr[:data.ptr[1]])
    print(data.y[:data.ptr[1]])
    print(data.winners[:data.ptr[1]])
    out = election_model(data)
    print(out[:data.ptr[1]])
    print("---")
    print(out[:data.ptr[1]][data.winners[:data.ptr[1]].bool()])
    print(out.size())

DataBatch(x=[11520, 2], edge_index=[2, 144000], edge_attr=[144000, 1], candidate_idxs=[11520], y=[1920], winners=[1920], batch=[11520], ptr=[129])
tensor([[1.0000],
        [0.9333],
        [0.8667],
        [0.8000],
        [0.7333],
        [0.6667],
        [0.6000],
        [0.5333],
        [0.4667],
        [0.4000],
        [0.3333],
        [0.2667],
        [0.2000],
        [0.1333],
        [0.0667],
        [1.0000],
        [0.9333],
        [0.8667],
        [0.8000],
        [0.7333],
        [0.6667],
        [0.6000],
        [0.5333],
        [0.4667],
        [0.4000],
        [0.3333],
        [0.2667],
        [0.2000],
        [0.1333],
        [0.0667],
        [1.0000],
        [0.9333],
        [0.8667],
        [0.8000],
        [0.7333],
        [0.6667],
        [0.6000],
        [0.5333],
        [0.4667],
        [0.4000],
        [0.3333],
        [0.2667],
        [0.2000],
        [0.1333],
        [0.0667],
        [1.0000],
        [0.9333],
       

In [59]:
for data in islice(eval_dataloader, 1):
    print(data)
    print(data.edge_attr[:data.ptr[1]])
    print(data.y[:data.ptr[1]])
    print(data.winners[:data.ptr[1]])
    print(data.candidate_idxs[:data.ptr[1]])
    print("---")
    out = strategy_model(data.edge_attr, data.edge_index, data.candidate_idxs)
    print(out.size())
    print(out[:data.ptr[1]])

DataBatch(x=[11520, 2], edge_index=[2, 144000], edge_attr=[144000, 1], candidate_idxs=[11520], y=[1920], winners=[1920], batch=[11520], ptr=[129])
tensor([[1.0000],
        [0.9333],
        [0.8667],
        [0.8000],
        [0.7333],
        [0.6667],
        [0.6000],
        [0.5333],
        [0.4667],
        [0.4000],
        [0.3333],
        [0.2667],
        [0.2000],
        [0.1333],
        [0.0667],
        [1.0000],
        [0.9333],
        [0.8667],
        [0.8000],
        [0.7333],
        [0.6667],
        [0.6000],
        [0.5333],
        [0.4667],
        [0.4000],
        [0.3333],
        [0.2667],
        [0.2000],
        [0.1333],
        [0.0667],
        [1.0000],
        [0.9333],
        [0.8667],
        [0.8000],
        [0.7333],
        [0.6667],
        [0.6000],
        [0.5333],
        [0.4667],
        [0.4000],
        [0.3333],
        [0.2667],
        [0.2000],
        [0.1333],
        [0.0667],
        [1.0000],
        [0.9333],
       

In [50]:
torch.save(election_model.state_dict(), "election_model")